## Prefect 2 Tutorial - Intro

In [1]:
from prefect import flow, task
import requests
import asyncio

In [2]:
!prefect version

Version:             2.1.1
API version:         0.8.0
Python version:      3.9.12
Git commit:          dc2ba222
Built:               Thu, Aug 18, 2022 10:18 AM
OS/Arch:             darwin/arm64
Profile:             default
Server type:         ephemeral
Server:
  Database:          sqlite
  SQLite version:    3.37.0


### Run a basic flow

The simplest way to begin with Prefect is to import flow and annotate your a Python function using the `@flow` decorator:

In [3]:
@task 
def add_one(x):
    return x + 1

@flow 
def my_flow():
    result = add_one(1) # return int

In [4]:
@flow 
def my_add_one_flow():
    state = add_one(1, return_state=True) # return State
    return state

In [5]:
state = my_add_one_flow()

15:49:59.836 | INFO    | prefect.engine - Created flow run 'adept-capuchin' for flow 'my-add-one-flow'
15:49:59.936 | INFO    | Flow run 'adept-capuchin' - Created task run 'add_one-3c3112ef-0' for task 'add_one'
15:49:59.937 | INFO    | Flow run 'adept-capuchin' - Executing 'add_one-3c3112ef-0' immediately...
15:49:59.971 | INFO    | Task run 'add_one-3c3112ef-0' - Finished in state Completed()
15:49:59.995 | INFO    | Flow run 'adept-capuchin' - Finished in state Completed('All states completed.')


In [6]:
state.result()

2

In [7]:
@flow
def my_favorite_function():
    print("This function doesn't do much")
    return 42

@flow
def my_favorite_function_flow():
    return my_favorite_function(return_state=True)

Running a Prefect workflow manually is as easy as calling the annotated function. In this case, we run the      `my_favorite_function()` snippet shown above:

In [8]:
state = my_favorite_function_flow()

15:50:00.414 | INFO    | prefect.engine - Created flow run 'economic-armadillo' for flow 'my-favorite-function-flow'
15:50:00.646 | INFO    | Flow run 'economic-armadillo' - Created subflow run 'arrogant-cricket' for flow 'my-favorite-function'
15:50:00.754 | INFO    | Flow run 'arrogant-cricket' - Finished in state Completed()
15:50:00.771 | INFO    | Flow run 'economic-armadillo' - Finished in state Completed('All states completed.')


This function doesn't do much


The first thing you'll notice is the messages surrounding the expected output, "This function doesn't do much".

By adding the `@flow` decorator to your function, function calls will create a flow run — the Prefect Orion orchestration engine manages task and flow state, including inspecting their progress, regardless of where your flow code runs.

For clarity in future tutorial examples, we may not show these messages in results except where they are relevant to the discussion.

Next, examine `state`, which shows the result returned by the `my_favorite_function()` flow.

In [9]:
state

Completed(message=None, type=COMPLETED, result=42)

*Flows return states*

> You may notice that this call did not return the number 42 but rather a Prefect State object. States are the basic currency of communication between Prefect clients and the Prefect API, and can be used to define the conditions for orchestration rules as well as an interface for client-side logic.

In this case, the state of `my_favorite_function()` is "Completed", with no further message details ("None" in this example).

If you want to see the data returned by the flow, access it via the `.result()` method on the State object.

In [10]:
state.result()

42

### Run flows with parameters

As with any Python function, you can pass arguments. The positional and keyword arguments defined on your flow function are called parameters. To demonstrate:

In [11]:
@task
def call_api(url):
    return requests.get(url).json()


In [12]:
@flow
def api_flow(url):
    return call_api(url, return_state=True)

You can pass any parameters needed by your flow function, and you can pass parameters on the `@flow` decorator for configuration as well. We'll cover that in a future tutorial.

For now, run the `call_api()` flow, passing a valid URL as a parameter. In this case, we're sending a POST request to an API that should return valid JSON in the response.

In [13]:
state = api_flow("http://time.jsontest.com/")

15:50:01.229 | INFO    | prefect.engine - Created flow run 'sandy-jellyfish' for flow 'api-flow'
15:50:01.296 | INFO    | Flow run 'sandy-jellyfish' - Created task run 'call_api-ded10bed-0' for task 'call_api'
15:50:01.297 | INFO    | Flow run 'sandy-jellyfish' - Executing 'call_api-ded10bed-0' immediately...
15:50:01.740 | INFO    | Task run 'call_api-ded10bed-0' - Finished in state Completed()
15:50:01.759 | INFO    | Flow run 'sandy-jellyfish' - Finished in state Completed('All states completed.')


Again, Prefect Orion automatically orchestrates the flow run. Again, print the state and note the "Completed" state matches what Prefect Orion prints in your terminal.

In [14]:
state.result()

{'date': '08-22-2022',
 'milliseconds_since_epoch': 1661147401488,
 'time': '05:50:01 AM'}

In [15]:
state

Completed(message=None, type=COMPLETED, result={'date': '08-22-2022', 'milliseconds_since_epoch': 1661147401488, 'time': '05:50:01 AM'})

### Error handling

What happens if the Python function encounters an error while your flow is running? To see what happens whenever our flow does not complete successfully, let's intentionally run the `call_api()` flow above with a bad value for the URL:

In [16]:
state_bad = api_flow("foo") # - exception doesn't get caught/handled properly by Prefect?

15:50:02.103 | INFO    | prefect.engine - Created flow run 'merry-wallaby' for flow 'api-flow'
15:50:02.294 | INFO    | Flow run 'merry-wallaby' - Created task run 'call_api-ded10bed-0' for task 'call_api'
15:50:02.295 | INFO    | Flow run 'merry-wallaby' - Executing 'call_api-ded10bed-0' immediately...
15:50:02.316 | ERROR   | Task run 'call_api-ded10bed-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/mjboothaus/code/github/mjboothaus/try-prefect2/.venv_dev_try-prefect2/lib/python3.9/site-packages/prefect/engine.py", line 1069, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "/Users/mjboothaus/code/github/mjboothaus/try-prefect2/.venv_dev_try-prefect2/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 56, in run_sync_in_worker_thread
    return await anyio.to_thread.run_sync(call, cancellable=True)
  File "/Users/mjboothaus/code/github/mjboothaus/try-prefect2/.venv_dev_try-prefect2/lib/

MissingSchema: Invalid URL 'foo': No scheme supplied. Perhaps you meant http://foo?

In this situation, the call to `requests.get()` encounters an exception, but the flow run still returns! The exception is captured by Orion, which continues to shut down the flow run normally.

However, in contrast to the 'COMPLETED' state, we now encounter a 'FAILED' state signaling that something unexpected happened during execution.

In [19]:
state_bad

NameError: name 'state_bad' is not defined

This behavior is consistent across flow runs and task runs and allows you to respond to failures in a first-class way — whether by configuring orchestration rules in the Orion backend (retry logic) or by directly responding to failed states in client code.

### Run a basic flow with tasks

Let's now add some tasks to a flow so that we can orchestrate and monitor at a more granular level.

A task is a function that represents a distinct piece of work executed within a flow. You don't have to use tasks — you can include all of the logic of your workflow within the flow itself. However, encapsulating your business logic into smaller task units gives you more granular observability, control over how specific tasks are run (potentially taking advantage of parallel execution), and reusing tasks across flows and sub-flows.

Creating and adding tasks follows the exact same pattern as for flows. Import task and use the `@task` decorator to annotate functions as tasks.

Let's take the previous `call_api()` example and move the actual HTTP request to its own task.

In [20]:
@task(name="cat-url")
def call_api(url):
    response = requests.get(url)
    print(f"Response: {response.status_code}")
    return response.json()


@flow(name="cat")
def api_flow(url):
    fact_json = call_api(url)
    return 

As you can see, we still call these tasks as normal functions and can pass their return values to other tasks. We can then call our flow function — now called `api_flow()` — just as before and see the printed output. Prefect manages all the relevant intermediate states.

In [21]:
state = api_flow("https://catfact.ninja/fact")

15:50:49.208 | INFO    | prefect.engine - Created flow run 'aspiring-raccoon' for flow 'cat'
15:50:49.382 | INFO    | Flow run 'aspiring-raccoon' - Created task run 'cat-url-ded10bed-0' for task 'cat-url'
15:50:49.382 | INFO    | Flow run 'aspiring-raccoon' - Executing 'cat-url-ded10bed-0' immediately...
15:50:50.105 | INFO    | Task run 'cat-url-ded10bed-0' - Finished in state Completed()
15:50:50.128 | INFO    | Flow run 'aspiring-raccoon' - Finished in state Completed('All states completed.')


Response: 200


And of course we can create tasks that take input from and pass input to other tasks.



In [22]:
@task(name="t-example3-1")
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task(name="t-example3-2")
def parse_fact(response):
    print(response["fact"])
    return 

@flow(name="f-example3")
def api_flow(url):
    fact_json = call_api(url)
    parse_fact(fact_json)
    return

In [23]:
state = api_flow("https://catfact.ninja/fact")

15:50:53.418 | INFO    | prefect.engine - Created flow run 'raspberry-orangutan' for flow 'f-example3'
15:50:53.487 | INFO    | Flow run 'raspberry-orangutan' - Created task run 't-example3-1-ded10bed-0' for task 't-example3-1'
15:50:53.488 | INFO    | Flow run 'raspberry-orangutan' - Executing 't-example3-1-ded10bed-0' immediately...
15:50:54.217 | INFO    | Task run 't-example3-1-ded10bed-0' - Finished in state Completed()
15:50:54.231 | INFO    | Flow run 'raspberry-orangutan' - Created task run 't-example3-2-6803447a-0' for task 't-example3-2'
15:50:54.232 | INFO    | Flow run 'raspberry-orangutan' - Executing 't-example3-2-6803447a-0' immediately...
15:50:54.258 | INFO    | Task run 't-example3-2-6803447a-0' - Finished in state Completed()
15:50:54.274 | INFO    | Flow run 'raspberry-orangutan' - Finished in state Completed('All states completed.')


200
The first cartoon cat was Felix the Cat in 1919. In 1940, Tom and Jerry starred in the first theatrical cartoon “Puss Gets the Boot.” In 1981 Andrew Lloyd Weber created the musical Cats, based on T.S. Eliot’s Old Possum’s Book of Practical Cats.


In [24]:
print(state)

[Completed(message=None, type=COMPLETED, result={'fact': 'The first cartoon cat was\xa0Felix the Cat\xa0in 1919. In 1940, Tom and Jerry starred in the first theatrical cartoon “Puss Gets the Boot.” In 1981 Andrew Lloyd Weber created the musical\xa0Cats, based on T.S. Eliot’s Old\xa0Possum’s Book of Practical Cats.', 'length': 245}), Completed(message=None, type=COMPLETED, result=None)]


Combining tasks with arbitrary Python code

> Notice in the above example that all of our Python logic is encapsulated within task functions. While there are many benefits to using Prefect in this way, it is not a strict requirement. Interacting with the results of your Prefect tasks requires an understanding of Prefect futures.

### Run a flow within a flow

Not only can you call tasks functions within a flow, but you can also call other flow functions! Flows that run within other flows are called sub-flows and allow you to efficiently manage, track, and version common multi-task logic.

Consider the following simple example:

In [25]:
@flow
def common_flow(config: dict):
    print("I am a subgraph that shows up in lots of places!")
    intermediate_result = 42
    return intermediate_result

@flow
def main_flow():
    # do some things
    # then call another flow function
    data = common_flow(config={})
    # do more things

Whenever we run `main_flow` as above, a new run will be generated for `common_flow` as well. Not only is this run tracked as a sub-flow run of `main_flow`, but you can also inspect it independently in the UI!

In [26]:
flow_state = main_flow()

15:51:23.136 | INFO    | prefect.engine - Created flow run 'invisible-zebra' for flow 'main-flow'
15:51:23.295 | INFO    | Flow run 'invisible-zebra' - Created subflow run 'beryl-dingo' for flow 'common-flow'
15:51:23.337 | INFO    | Flow run 'beryl-dingo' - Finished in state Completed()
15:51:23.356 | INFO    | Flow run 'invisible-zebra' - Finished in state Completed('All states completed.')


I am a subgraph that shows up in lots of places!


You can confirm this for yourself by spinning up the UI using the `prefect orion start` CLI command from your terminal:


In [ ]:
!prefect start

### Asynchronous functions

Even asynchronous functions work with Prefect! Here's a variation of the previous examples that makes the API request as an async operation:

In [29]:
@task(name="async-url")
async def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()


@flow(name="async-flow")
async def async_flow(url):
    fact_json = await call_api(url)
    return 

/Users/mjboothaus/code/github/mjboothaus/try-prefect2/.venv_dev_try-prefect2/lib/python3.9/site-packages/prefect/tasks.py:166: UserWarning: A task named 'async-url' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_70180/393539765.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/mjboothaus/code/github/mjboothaus/try-prefect2/.venv_dev_try-prefect2/lib/python3.9/site-packages/prefect/flows.py:176: UserWarning: A flow named 'async-flow' and defined at '/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_70180/393539765.py:8' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


If we run this in the REPL, the output looks just like previous runs.

In [30]:
# Following doesn't seem to run in Jupyter

asyncio.run(async_flow("https://catfact.ninja/fact"))

RuntimeError: asyncio.run() cannot be called from a running event loop